In [1]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('Task4.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.dropna(subset=['Survived'],inplace=True)
X = data[['Pclass', 'Sex', 'Age', 'Fare']]
y = data['Survived']
X['Pclass'] = X['Pclass'].astype('str')
X['Fare'].fillna(0,inplace=True)
X['Age'].fillna(0,inplace=True)
def get_categorical_indicies(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_categorical_indicies(X)

def convert_cats(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
convert_cats(X)

<ipython-input-3-824d0b336b68>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Pclass'] = X['Pclass'].astype('str')
<ipython-input-3-824d0b336b68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Fare'].fillna(0,inplace=True)
<ipython-input-3-824d0b336b68>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'].fillna(0,inplace=True)
<ipython-input-3-824d0b336b68>:29: Se

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.2,
                                                 random_state=101,
                                                 stratify=y)
print('Test Survival Rate:',y_test.sum()/y_test.count())

Test Survival Rate: 0.3854748603351955


In [5]:
train_df = pd.concat([X,y],axis=1)
survived = train_df[train_df['Survived']==1]
deceased = train_df[train_df['Survived']==0]
deceased = deceased.sample(n=len(survived), random_state=101)
train_df = pd.concat([survived,deceased],axis=0)
X_train = train_df.drop('Survived',axis=1)
y_train = train_df['Survived']
train_dataset = cb.Pool(X_train,y_train,
                        cat_features=categorical_indicies)
test_dataset = cb.Pool(X_test,y_test,
                       cat_features=categorical_indicies)


model = cb.CatBoostClassifier(loss_function='Logloss',
                              eval_metric='Accuracy')
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations': [50, 100, 150]}


model.grid_search(grid,train_dataset)
pred = model.predict(X_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
86:	learn: 0.8062157	test: 0.7445255	best: 0.7810219 (1)	total: 176ms	remaining: 26.3ms
87:	learn: 0.8062157	test: 0.7299270	best: 0.7810219 (1)	total: 177ms	remaining: 24.1ms
88:	learn: 0.8043876	test: 0.7299270	best: 0.7810219 (1)	total: 178ms	remaining: 22ms
89:	learn: 0.8025594	test: 0.7299270	best: 0.7810219 (1)	total: 179ms	remaining: 19.9ms
90:	learn: 0.8043876	test: 0.7299270	best: 0.7810219 (1)	total: 181ms	remaining: 17.9ms
91:	learn: 0.8025594	test: 0.7299270	best: 0.7810219 (1)	total: 182ms	remaining: 15.8ms
92:	learn: 0.8025594	test: 0.7445255	best: 0.7810219 (1)	total: 183ms	remaining: 13.8ms
93:	learn: 0.8007313	test: 0.7445255	best: 0.7810219 (1)	total: 188ms	remaining: 12ms
94:	learn: 0.8007313	test: 0.7445255	best: 0.7810219 (1)	total: 189ms	remaining: 9.96ms
95:	learn: 0.8062157	test: 0.7299270	best: 0.7810219 (1)	total: 191ms	remaining: 7.94ms
96:	learn: 0.8062157	test: 0.7299270	best: 0.7810219 (1)

In [6]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.77      0.89      0.82       110
           1       0.76      0.57      0.65        69

    accuracy                           0.77       179
   macro avg       0.77      0.73      0.74       179
weighted avg       0.77      0.77      0.76       179

